# Spotify API Scrape

In [2]:
# run this once
# !pip install spotipy --upgrade

In [3]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import numpy as np
import json
import os

In [4]:
with open("../api_cred.json") as json_file:
    creds = json.load(json_file)

In [5]:
client_id = creds['client_id']
client_secret = creds['client_secret']
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [41]:
def load_cached_requests(fp):
    if os.path.exists(fp):
        with open(fp) as json_file:
            ret_dic = json.load(json_file)
            print("Loaded cached data at:",fp)
    else:
        print("Failed to load cached data at:",fp)
        ret_dic = {}
    return ret_dic

def save_cached_requests(save_dic, save_dir,fname):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    with open(save_dir+fname,'w+') as outfile:
        json.dump(save_dic,outfile)
    print("Requests Cached Successfully at:", save_dir+fname)

def save_csvs(data, save_dir, fname):
    for genre in data:
        genre_save_dir = save_dir.format(genre)
        genre_fname = fname.format(genre)
        if not os.path.exists(genre_save_dir):
            os.makedirs(genre_save_dir)
        data[genre].to_csv(genre_save_dir + genre_fname, index=False)
        print("CSV Saved at:", genre_save_dir + genre_fname)

In [7]:
def get_playlists_data_from_category(cat_id,master_playlist_dic):
    # api call
    search_result = sp.category_playlists(category_id=cat_id,country="US")
    playlist_ids = []
    for playlist in search_result['playlists']['items']:
        playlist_ids.append(playlist['id'])
    playlists_data = []
    for playlist_id in playlist_ids:
        # avoids api call using cached request, could hold outdated info if playlist is changed recently
        if playlist_id in master_playlist_dic.keys():
            playlist_data = master_playlist_dic[playlist_id]
        else:
            # api call
            playlist_data = sp.playlist(playlist_id)
            master_playlist_dic[playlist_id] = playlist_data
        playlists_data.append(playlist_data)
    return playlists_data

In [8]:
def get_track_data_from_playlists(playlists_data, master_track_dic, master_audio_features_dic):
    ret_df = pd.DataFrame()
    for cur_playlist_data in playlists_data: # only doing two for api reasons currently

        # can add more features here (album, artist, etc..) MOVED TO FUNCTION USING CACHED DICS
        cur_playlist_track_ids = []
        for track in cur_playlist_data['tracks']['items']:
            if track['track']:
                cur_track_id = track['track']['id']
                if cur_track_id != None:
                    # caches track metadata if not cached
                    if cur_track_id not in master_track_dic.keys():
                        master_track_dic[cur_track_id] = track
                    cur_playlist_track_ids.append(cur_track_id)
                
        # making a list of non cached track audio features
        not_cached_ids = []
        for track_id in cur_playlist_track_ids:
            if track_id not in master_audio_features_dic.keys():
                not_cached_ids.append(track_id)

        # get audio features of non cached tracks
        if len(not_cached_ids) > 0:
            print("Making",len(not_cached_ids),"API Calls")
            # api call
            not_cached_data = sp.audio_features(not_cached_ids)
            # caching the non cached track audio features
            for i in range(len(not_cached_data)):
                track_feats = not_cached_data[i]
                if track_feats:
                    master_audio_features_dic[track_feats['id']] = track_feats
                else:
                    master_audio_features_dic[not_cached_ids[i]] = None

        # iterating current playlist tracks and accessing the cached audio features
        playlist_track_feats = []
        for track_id in cur_playlist_track_ids:
            cur_feats = master_audio_features_dic[track_id]
            if cur_feats:
                playlist_track_feats.append(cur_feats)

        # building return df
        cur_playlist_df = pd.DataFrame(playlist_track_feats)
        ret_df = pd.concat([ret_df, cur_playlist_df])
    ret_df = ret_df.reset_index(drop=True)
    return ret_df

In [57]:
def get_genre_data(target_genres, cache_dir, csv_dir, save_data=True):
    master_track_fname = "track_data.json"
    master_playlist_fname = "playlist_data.json"
    master_audio_features_fname = "track_audio_features.json"
    master_track_dic = load_cached_requests(cache_dir + master_track_fname)
    master_playlist_dic = load_cached_requests(cache_dir + master_playlist_fname)
    master_audio_features_dic = load_cached_requests(cache_dir + master_audio_features_fname)
    
    genres_data = {}
    genres_data["all"] = pd.DataFrame()
    for genre in target_genres:
        genre_data = get_playlists_data_from_category(genre, master_playlist_dic)
        genre_df = get_track_data_from_playlists(genre_data, master_track_dic, master_audio_features_dic)
        genre_df["genre"] = genre
        genres_data[genre] = genre_df
        genres_data["all"] = pd.concat([genre_df, genres_data["all"]])
    genres_data["all"] = genres_data["all"].drop_duplicates().reset_index(drop=True)
    
    save_cached_requests(master_track_dic,cache_dir,master_track_fname)
    save_cached_requests(master_playlist_dic,cache_dir,master_playlist_fname)
    save_cached_requests(master_audio_features_dic,cache_dir,master_audio_features_fname)
    
    if save_data:
        csv_fname = "{}_metadata.csv"
        save_csvs(genres_data, csv_dir, csv_fname)
        
    return genres_data

In [58]:
target_genres = ["kpop", "pop", "rock"]
genres_data = get_genre_data(target_genres,
                             cache_dir="../data/local/cached_requests/",
                             csv_dir="../data/test/{}/metadata/")

Loaded cached data at: ../data/local/cached_requests/track_data.json
Loaded cached data at: ../data/local/cached_requests/playlist_data.json
Loaded cached data at: ../data/local/cached_requests/track_audio_features.json
Requests Cached Successfully at: ../data/local/cached_requests/track_data.json
Requests Cached Successfully at: ../data/local/cached_requests/playlist_data.json
Requests Cached Successfully at: ../data/local/cached_requests/track_audio_features.json
CSV Saved at: ../data/test/all/metadata/all_metadata.csv
CSV Saved at: ../data/test/kpop/metadata/kpop_metadata.csv
CSV Saved at: ../data/test/pop/metadata/pop_metadata.csv
CSV Saved at: ../data/test/rock/metadata/rock_metadata.csv
